In [ ]:
! git clone --recursive https://github.com/castorini/pygaggle.git
%cd pygaggle/

Cloning into 'pygaggle'...
remote: Enumerating objects: 1557, done.
remote: Counting objects: 100% (629/629), done.
remote: Compressing objects: 100% (220/220), done.
remote: Total 1557 (delta 525), reused 436 (delta 409), pack-reused 928
Receiving objects: 100% (1557/1557), 509.74 KiB | 13.07 MiB/s, done.
Resolving deltas: 100% (998/998), done.
Submodule 'tools' (https://github.com/castorini/anserini-tools.git) registered for path 'tools'
Cloning into '/content/pygaggle/tools'...
remote: Enumerating objects: 723, done.        
remote: Counting objects: 100% (480/480), done.        
remote: Compressing objects: 100% (418/418), done.        
remote: Total 723 (delta 74), reused 453 (delta 61), pack-reused 243        
Receiving objects: 100% (723/723), 57.78 MiB | 12.01 MiB/s, done.
Resolving deltas: 100% (158/158), done.
Submodule path 'tools': checked out '808f48711b5e172da6aec8b1855518c8ea65489f'
/content/pygaggle


In [ ]:
!pip install /content/pygaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/pygaggle
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 KB 89.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 43.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.1/137.1 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.7/489.7 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 105.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 KB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 90.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 11.9 MB/s eta 0:00:00


In [ ]:
from pygaggle.rerank.base import Query, Text
from pygaggle.rerank.transformer import MonoT5

reranker =  MonoT5('castorini/monot5-base-med-msmarco')

2023-03-14 17:06:51 [INFO] utils: NumExpr defaulting to 2 threads.
2023-03-14 17:06:54 [INFO] loader: Loading faiss with AVX2 support.
2023-03-14 17:06:54 [INFO] loader: Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'")
2023-03-14 17:06:54 [INFO] loader: Loading faiss.
2023-03-14 17:06:54 [INFO] loader: Successfully loaded faiss.


Downloading:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd /content/drive/My Drive/NLPWork/

/content/drive/My Drive/NLPWork


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import json

In [ ]:
results = {}
def get_relevant_sentences(cs, num_tfidf):
    global results

    for pmcid in cs:
        print(pmcid)
        if pmcid.startswith('0'):
            curr_ref_file = pmcid.split('_')[1]
        else:
            curr_ref_file = pmcid
            
        CURR_PMC = pmcid

        with open(f'Sentences/{curr_ref_file}.json') as f:
            sentences = json.load(f)

        vectorizer = TfidfVectorizer()
        X = vectorizer.fit_transform(sentences)

        results[CURR_PMC] = {}

        idx = 0
        for file, d in cs[pmcid].items():
            if len(d['support']) == 0:
                continue
            claim = d['context']
            # print(f'Claim {idx}')
            idx += 1
            
            y = vectorizer.transform([claim])
            cosine_similarities = linear_kernel(y, X).flatten()
            related_docs_indices = cosine_similarities.argsort()[::-1][:num_tfidf]
            texts = [Text(sentences[i], {'sent_idx' : i}, 0) for i in related_docs_indices]
            query = Query(claim)
            reranked = reranker.rerank(query, texts)
            current_sentences = []
            # Print out reranked results:
            for i in range(0, 10):
                current_sentences.append(reranked[i].text)
            results[CURR_PMC][file] = current_sentences

In [ ]:
def calculate_recall(cs, results):
    def overlap(start, end, s, e):
        return start <= e and s <= end

    for pmcid in results:
        if pmcid.startswith('0'):
            curr_ref_file = pmcid.split('_')[1]
        else:
            curr_ref_file = pmcid
        with open(f'Articles/{curr_ref_file}.txt', encoding="utf-8") as f:
            content = f.read()
        for file in results[pmcid]:
            indices = [[content.find(s.strip()), content.find(s.strip())+len(s.strip())] for s in results[pmcid][file]]
            count = 0
            support_indices = [[content.find(s.strip()), content.find(s.strip())+len(s.strip())] for s in cs[pmcid][file]['support']]
            for start, end in support_indices:
                for s, e in indices:
                    if overlap(start, end, s, e):
                        count += 1
            cs[pmcid][file]['count'] = count
    res = 0
    total = 0
    for pmcid in cs:
        for file in cs[pmcid]:
            if 'count' in cs[pmcid][file]:
                res += (cs[pmcid][file]['count'] / len(cs[pmcid][file]['support']))
                total += 1

    return res/total

In [ ]:
with open('recon.json') as f:
    cs = json.load(f)

with open('recon_no_title.json') as f:
    cst = json.load(f)

with open('Eugene_ContextAndSupport.json') as f:
    cs1 = json.load(f)

with open('Eugene_ContextAndSupport_No_Title.json') as f:
    cs2 = json.load(f)

with open('Degaulle_ContextAndSupport.json') as f:
    cs3 = json.load(f)

with open('Degaulle_ContextAndSupport_No_Title.json') as f:
    cs4 = json.load(f)

# 50 TF-IDF

## Eugene - With Title

In [ ]:
results = {}
get_relevant_sentences(cs1, 50)

PMC2811155
PMC2848616
PMC3442244
PMC3522098
PMC3090745
PMC3650111
PMC3096954
PMC2958248
PMC3179858
PMC3607626


In [ ]:
calculate_recall(cs1, results)

0.3228878648233487

## Eugene - No Title

In [ ]:
results = {}
get_relevant_sentences(cs2, 50)

PMC2811155
Claim 0
Claim 1
Claim 2
Claim 3
Claim 4
Claim 5
Claim 6
Claim 7
Claim 8
Claim 9
Claim 10
Claim 11
Claim 12
Claim 13
Claim 14
Claim 15
Claim 16
Claim 17
Claim 18
Claim 19
Claim 20
Claim 21
Claim 22
Claim 23
Claim 24
Claim 25
Claim 26
PMC2848616
Claim 0
Claim 1
Claim 2
Claim 3
Claim 4
Claim 5
Claim 6
Claim 7
Claim 8
Claim 9
Claim 10
Claim 11
Claim 12
Claim 13
Claim 14
Claim 15
Claim 16
Claim 17
Claim 18
Claim 19
Claim 20
Claim 21
Claim 22
Claim 23
Claim 24
PMC3442244
Claim 0
Claim 1
Claim 2
Claim 3
Claim 4
Claim 5
Claim 6
Claim 7
Claim 8
Claim 9
Claim 10
Claim 11
Claim 12
PMC3522098
Claim 0
Claim 1
Claim 2
PMC3090745
Claim 0
Claim 1
Claim 2
Claim 3
Claim 4
Claim 5
Claim 6
PMC3650111
Claim 0
PMC3096954
Claim 0
Claim 1
Claim 2
Claim 3
Claim 4
Claim 5
Claim 6
Claim 7
Claim 8
Claim 9
Claim 10
Claim 11
Claim 12
Claim 13
Claim 14
Claim 15
Claim 16
Claim 17
Claim 18
Claim 19
Claim 20
PMC2958248
Claim 0
Claim 1
Claim 2
Claim 3
Claim 4
Claim 5
Claim 6
Claim 7
Claim 8
Claim 9
Claim 10
C

In [ ]:
calculate_recall(cs2, results)

0.5246376811594203

## Degaulle

In [ ]:
results = {}
get_relevant_sentences(cs3, 50)
calculate_recall(cs3, results)

010_PMC3650111
004_PMC3090745
001_PMC2811155
007_PMC3442244
005_PMC3096954
006_PMC3179858
002_PMC2848616
003_PMC2958248
008_PMC3522098
009_PMC3607626


0.6155367231638419

In [ ]:
results = {}
get_relevant_sentences(cs4, 50)
calculate_recall(cs4, results)

010_PMC3650111
004_PMC3090745
001_PMC2811155
007_PMC3442244
005_PMC3096954
006_PMC3179858
002_PMC2848616
003_PMC2958248
008_PMC3522098
009_PMC3607626


0.6529761904761905

# 100 TF-IDF

In [ ]:
results = {}
get_relevant_sentences(cs1, 100)
calculate_recall(cs1, results)

PMC2811155
PMC2848616
PMC3442244
PMC3522098
PMC3090745
PMC3650111
PMC3096954
PMC2958248
PMC3179858
PMC3607626


0.31751152073732714

In [ ]:
results = {}
get_relevant_sentences(cs2, 100)
calculate_recall(cs2, results)

PMC2811155
PMC2848616
PMC3442244
PMC3522098
PMC3090745
PMC3650111
PMC3096954
PMC2958248
PMC3179858
PMC3607626


0.5198067632850241

In [ ]:
results = {}
get_relevant_sentences(cs3, 100)
calculate_recall(cs3, results)

010_PMC3650111
004_PMC3090745
001_PMC2811155
007_PMC3442244
005_PMC3096954
006_PMC3179858
002_PMC2848616
003_PMC2958248
008_PMC3522098
009_PMC3607626


0.6371468926553673

In [ ]:
results = {}
get_relevant_sentences(cs4, 100)
calculate_recall(cs4, results)

010_PMC3650111
004_PMC3090745
001_PMC2811155
007_PMC3442244
005_PMC3096954
006_PMC3179858
002_PMC2848616
003_PMC2958248
008_PMC3522098
009_PMC3607626


0.6757440476190476

In [ ]:
results = {}
get_relevant_sentences(cs, 100)
calculate_recall(cs, results)

010_PMC3650111
004_PMC3090745
001_PMC2811155
007_PMC3442244
005_PMC3096954
006_PMC3179858
002_PMC2848616
003_PMC2958248
008_PMC3522098
009_PMC3607626


0.5405833333333333

In [ ]:
results = {}
get_relevant_sentences(cst, 100)
calculate_recall(cst, results)

010_PMC3650111
004_PMC3090745
001_PMC2811155
007_PMC3442244
005_PMC3096954
006_PMC3179858
002_PMC2848616
003_PMC2958248
008_PMC3522098
009_PMC3607626


0.600089605734767

# 250 TF-IDF

In [ ]:
results = {}
get_relevant_sentences(cs1, 250)
calculate_recall(cs1, results)

PMC2811155
PMC2848616
PMC3442244
PMC3522098
PMC3090745
PMC3650111
PMC3096954
PMC2958248
PMC3179858
PMC3607626


0.32519201228878647

In [ ]:
results = {}
get_relevant_sentences(cs2, 250)
calculate_recall(cs2, results)

PMC2811155
PMC2848616
PMC3442244
PMC3522098
PMC3090745
PMC3650111
PMC3096954
PMC2958248
PMC3179858
PMC3607626


0.5318840579710145

In [ ]:
results = {}
get_relevant_sentences(cs3, 250)
calculate_recall(cs3, results)

010_PMC3650111
004_PMC3090745
001_PMC2811155
007_PMC3442244
005_PMC3096954
006_PMC3179858
002_PMC2848616
003_PMC2958248
008_PMC3522098
009_PMC3607626


0.6371468926553673

In [ ]:
results = {}
get_relevant_sentences(cs4, 250)
calculate_recall(cs4, results)

010_PMC3650111
004_PMC3090745
001_PMC2811155
007_PMC3442244
005_PMC3096954
006_PMC3179858
002_PMC2848616
003_PMC2958248
008_PMC3522098
009_PMC3607626


0.6757440476190476

In [ ]:
results = {}
get_relevant_sentences(cst, 250)
calculate_recall(cst, results)

010_PMC3650111
004_PMC3090745
001_PMC2811155
007_PMC3442244
005_PMC3096954
006_PMC3179858
002_PMC2848616
003_PMC2958248
008_PMC3522098
009_PMC3607626


0.6312724014336917